In [1]:
%load_ext autoreload
%autoreload 2

#per la riproducibilità
#%env CUBLAS_WORKSPACE_CONFIG=":4096:8"

import os
import itertools
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from IPython.display import Markdown as md
from multiprocessing import Pool
import time
from tqdm import tqdm
import networkx as nx
from networkx import stochastic_block_model

import torch_geometric
from graph_generation import GenerateGraph, perturb_nx_graph
from models import GCN, view_parameters, new_parameters, modify_parameters, Inits, new_parameters_linears
from train import Trainer, Dataset
from embedding import Embedding
from config_valid import Config, TrainingMode
import experiments
from experiments import Experiments, all_seeds
from plot_funcs import (plot_dim1, plot_dimN, plot_correlation_error, plot_metrics, plot_node_emb_1D, plot_node_emb_nD, scatter_node_emb, 
                        plot_graph_emb_1D, plot_graph_emb_nD, plot_data_degree_sequence, plot_corr_epoch, plot_ripetizioni_stesso_trial, 
                        plot_onlyloss_ripetizioni_stesso_trial,plot_onlyloss_ripetizioni_stesso_trial_superimposed, Data2Plot, plot_weights_multiple_hist)
from plot_model import plot_model
from utils_embednet import array_wo_outliers, plot_grafo, plot_grafo2
from Inspect import Inspect
#import model_graph_ae


import torch
from torch_geometric import nn
from torch_geometric.loader import DataLoader
device = torch.device('cuda')

from scipy import stats
from sklearn.metrics import log_loss

import pickle 
from plt_parameters import init_params, get_colors_to_cycle_rainbow8, get_colors_to_cycle_rainbowN
init_params()
rootsave = Path("output_plots/")

def make_video_parallel_static():
    experiments.graph_embedding_per_epoch = xp.trainer.graph_embedding_per_epoch
    experiments.node_embedding_per_epoch = xp.trainer.node_embedding_per_epoch
    experiments.dataset = xp.trainer.dataset
    experiments.loss_list = xp.trainer.test_loss_list
    experiments.exp_config = xp.trainer.config_class
    experiments.dataset_type = xp.trainer.gg.graphtype
    num_emb_neurons = xp.trainer.model.convs[-1].out_channels
    experiments.trainmode = xp.trainer.config_class.modo
    #experiments.num_classes = xp.trainer.config_class.num_classes
    experiments.embedding_dimension = num_emb_neurons
    #my_list = my_log_lista=list(range(20)) + list(range(20,100,4)) + list(range(100,500, 15)) + list(range(500, 5000, 40))
    nomefile = xp.make_video(skip=1, fromfiles=True, custom_list=True, seq_colors=True)
    return nomefile


def run_grid_w_gif(xp):
    nomifilesgif = []
    k = 0
    for c in xp.gc.configs:  
        print(f'Run {k + 1}\t\t exp name: {c.unique_train_name}')
        # all_seeds()
        xp.trainer.reinit_conf(c)
        xp.just_train()
        embedding_class = xp.embedding()
        num_emb_neurons = xp.trainer.model.convs[-1].out_channels
        trainmode = xp.trainer.config_class.modo
        embedding_class.get_metrics(num_emb_neurons, trainmode)    
        nomefile = make_video_parallel_static()
        nomifilesgif.append(nomefile)
        k += 1
    return nomifilesgif

In [2]:
config_file = "configurations/Final1.yml"
num_nodi = 200
c = Config(config_file)
#c.conf['graph_dataset']['Num_nodes'] = [num_nodi]
#c.conf['graph_dataset']['list_exponents'] = [-2.5]
c.conf['model']['autoencoder'] = True
c.conf['model']['autoencoder_graph_ae'] = False
diz_trials = {'graph_dataset.ERmodel': [False], 
              'graph_dataset.confmodel': [False], 
              'graph_dataset.sbm': [False], 
              'graph_dataset.real_dataset': [True],
              #'graph_dataset.Num_nodes': [[num_nodi]*5, [num_nodi]*3],  # per lo SBM: num nodi * num comunità 
              'model.GCNneurons_per_layer': [#[1, 32, 16, len(c.conf['graph_dataset']['list_exponents'])],
                                            #[1, 32, 16, len(c.conf['graph_dataset']['list_p'])],
                                            #[1, 32, 16, len(c.conf['graph_dataset']['community_probs'])],
                                            [1, 32, 32, 32, 1]
                                           ],
             #'model.init_weights': ['xavier_normal'],# 'eye'],
             'model.freezeGCNlayers': [False],
             'model.last_layer_dense': [False],
             } 

xp = Experiments(diz_trials=diz_trials, rootsave=rootsave, config_class=c, reset_all_seeds=False, verbose=True)

Autoencoder model
0 configurazioni saltate su 1, farò i seguenti 1 training: 
REDDIT-BINARY___Classi2_nodi_grafiXtipo_mode1_layers§1-32-32-32-1§_initw-eye_lr0.002_GCNfreezedFalse


### Con Reddit ho questo errore:

In [3]:
xp.GS_simple_experiments(do_plot=False)

Run 1 		 exp name: REDDIT-BINARY___Classi2_nodi_grafiXtipo_mode1_layers§1-32-32-32-1§_initw-eye_lr0.002_GCNfreezedFalse
Verrà usato il dataset REDDIT-BINARY
Loading Dataset...


100%|████████████████████████████████████████████████████████████████████████████████████████| 250/250 [08:00<00:00,  1.92s/it]


test accuracy finale: 0


IndexError: index 600 is out of bounds for axis 0 with size 600

In [ ]:
xp.gc.configs

In [ ]:
c = xp.gc.configs[0]
xp.trainer.reinit_conf(c)
xp.trainer.init_all()

In [ ]:
xp.trainer.launch_training()

In [ ]:
xp.trainer.test_loss_list

In [ ]:
embedding_class = xp.embedding()